## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.33,63,97,4.09,overcast clouds
1,1,Barrow,US,71.2906,-156.7887,-11.18,77,40,10.36,scattered clouds
2,2,Faya,SA,18.3851,42.4509,57.31,81,14,5.03,few clouds
3,3,Hobart,AU,-42.8794,147.3294,58.05,94,75,1.01,broken clouds
4,4,Tuktoyaktuk,CA,69.4541,-133.0374,-2.20,84,100,19.57,light snow


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.33,63,97,4.09,overcast clouds
7,7,Cabo San Lucas,MX,22.8909,-109.9124,83.52,66,20,5.75,few clouds
8,8,Apac,UG,1.9756,32.5386,79.23,35,99,2.35,overcast clouds
11,11,Vaini,TO,-21.2000,-175.2000,80.06,78,100,11.12,light rain
18,18,Puerto Escondido,MX,15.8500,-97.0667,85.03,70,0,6.91,clear sky
21,21,Manggar,ID,-2.8833,108.2667,78.13,80,100,5.48,overcast clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,85.96,71,100,9.19,overcast clouds
25,25,Arraial Do Cabo,BR,-22.9661,-42.0278,85.96,70,40,11.50,thunderstorm
26,26,Butaritari,KI,3.0707,172.7902,81.91,76,55,22.10,light rain
27,27,Sao Jose Da Coroa Grande,BR,-8.8978,-35.1478,81.70,74,100,9.10,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows in the preferred_cities_df
clean_df = preferred_cities_df

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.33,overcast clouds,-23.1203,-134.9692,
7,Cabo San Lucas,MX,83.52,few clouds,22.8909,-109.9124,
8,Apac,UG,79.23,overcast clouds,1.9756,32.5386,
11,Vaini,TO,80.06,light rain,-21.2000,-175.2000,
18,Puerto Escondido,MX,85.03,clear sky,15.8500,-97.0667,
21,Manggar,ID,78.13,overcast clouds,-2.8833,108.2667,
22,Puerto Ayora,EC,85.96,overcast clouds,-0.7393,-90.3518,
25,Arraial Do Cabo,BR,85.96,thunderstorm,-22.9661,-42.0278,
26,Butaritari,KI,81.91,light rain,3.0707,172.7902,
27,Sao Jose Da Coroa Grande,BR,81.70,overcast clouds,-8.8978,-35.1478,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()
# No rows without a name
clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.33,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Cabo San Lucas,MX,83.52,few clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
8,Apac,UG,79.23,overcast clouds,1.9756,32.5386,Double Duez Apac
11,Vaini,TO,80.06,light rain,-21.2000,-175.2000,Keleti Beach Resort
18,Puerto Escondido,MX,85.03,clear sky,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
21,Manggar,ID,78.13,overcast clouds,-2.8833,108.2667,Guest Hotel
22,Puerto Ayora,EC,85.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
25,Arraial Do Cabo,BR,85.96,thunderstorm,-22.9661,-42.0278,Pousada Porto Praia
26,Butaritari,KI,81.91,light rain,3.0707,172.7902,Isles Sunset Lodge
27,Sao Jose Da Coroa Grande,BR,81.70,overcast clouds,-8.8978,-35.1478,Crôa Mares Hotel


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))